In [8]:
# Required by Google Colab
!pip install qiskit qiskit_nature pyscf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 33.6 MB/s eta 0:00:00


### Define electronic molecule of $BeH_{2}$

In [40]:
import time
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

# Define molecule BeH2
distance = 1.7
molecule = Molecule(geometry=[['H', [0., 0., -distance]],
                              ['Be', [0., 0., 0.]],
                              ['H', [0., 0., distance]],
                              ], charge=0, multiplicity=1)

# set up a PySCF driver at bondlength 1.7 Angstrom (distance)
driver = ElectronicStructureMoleculeDriver(molecule, basis='sto6g', driver_type=ElectronicStructureDriverType.PYSCF)

freezeCoreTransformer = FreezeCoreTransformer(True)
problem = ElectronicStructureProblem(driver, transformers=[freezeCoreTransformer])

# JordanWignerMapper()
mapper = ParityMapper()

qubit_converter = QubitConverter(mapper, two_qubit_reduction=True, z2symmetry_reduction='auto')

### VQE

In [41]:
%%time
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SLSQP
from qiskit_nature.algorithms import VQEUCCFactory, VQEUVCCFactory

params_history = []
def callback(xk):
    return params_history.append(xk)

optimizer = SLSQP(maxiter=50, callback=callback)
backend_name = 'aer_simulator_statevector' # Wall time: maxiter=1 -> 7.77 s, maxiter=50 -> 16.9 s
quantum_instance = QuantumInstance(backend = Aer.get_backend(backend_name))
solver = VQEUCCFactory(quantum_instance,optimizer=optimizer)
gs_solver = GroundStateEigensolver(qubit_converter, solver)
result = gs_solver.solve(problem)
print(result)

<timed exec>:14: DeprecationWarning: VQEUCCFactory Constructor: quantum_instance is no longer a positional argument as of version 0.4 and will be removed no sooner than 3 months after the release. Instead use it as a keyword argument


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -18.336179293342
  - computed part:      -3.497517988673
  - FreezeCoreTransformer extracted energy part: -14.838661304669
~ Nuclear repulsion energy (Hartree): 2.6458860546
> Total ground state energy (Hartree): -15.690293238742
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [None  None  None]
    - computed part:      [None  None  None]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [None  None  None]  Total: None
                 (debye): [None  None  None]  Total: None
 
CPU times: user 14.8 s, sys: 4.67 s, total: 19.5 s
Wall time: 15 s


In [24]:
problem.second_q_ops()

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j)), ('+_4 -_4', (1+0j)), ('+_5 -_5', (1+0j)), ('+_6 -_6', (1+0j)), ('+_7 -_7', (1+0j)), ('+_8 -_8', (1+0j)), ('+_9...)], register_length=12, display_format='sparse'),
 'ElectronicEnergy': FermionicOp([('+_0 -_0', -1.3480715508364554), ('+_0 -_4', -0.0027383488243007998), ('+_1 -_1', -1.3360288709735333), ('+_1 -_5', -0.02510993760317797), ('+_2 -_2', -1.0673997491942597), ('+_3 -_3', -1.06739...)], register_length=12, display_format='sparse'),
 'DipoleMomentX': FermionicOp([('+_0 -_2', 0.2055766671543245), ('+_0 -_3', 0.9426939302814478), ('+_2 -_0', 0.2055766671543245), ('+_2 -_4', 0.18346956596618358), ('+_3 -_0', 0.9426939302814477), ('+_3 -_4', 0.84131943873696...)], register_length=12, display_format='sparse'),
 'DipoleMomentY': FermionicOp([('+_0 -_2', 0.9426939302814474), ('+_0 -_3', -0.2055766671543248), ('+_2 -_0', 0.9426939302814474), ('+_2 -_4', 0.8413194387369686)

### Qubits tapering with Z2 symmetries reduction

In [43]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

#driver = PySCFDriver()
driver = PySCFDriver(
    atom=f"H 0 0 {-distance}; Be 0 0 0; H 0 0 {distance}",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

problem = driver.run()
fermionic_op = problem.hamiltonian.second_q_op()

In [45]:
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper

#mapper = JordanWignerMapper()
#converter = QubitConverter(mapper)
#qubit_op = converter.convert(fermionic_op)
#print(qubit_op) # 14 qubits

from qiskit_nature.second_q.mappers import ParityMapper

#mapper = ParityMapper()
#converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
#qubit_op = converter.convert(fermionic_op, num_particles=problem.num_particles)
#print(qubit_op) # 12 qubits

#converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction="auto")
#qubit_op = converter.convert(fermionic_op, sector_locator=problem.symmetry_sector_locator)
#print(qubit_op) # 9 qubits

#converter = QubitConverter(BravyiKitaevMapper(), z2symmetry_reduction="auto")
#qubit_op = converter.convert(fermionic_op, sector_locator=problem.symmetry_sector_locator)
#print(qubit_op) # 9 qubits

converter = QubitConverter(ParityMapper(), z2symmetry_reduction="auto")
qubit_op = converter.convert(fermionic_op, sector_locator=problem.symmetry_sector_locator)
print(qubit_op) # 9 qubits


-11.599689949681562 * IIIIIIIII
+ 2.2158249643468877 * IIIIIIIIZ
+ 0.029071696307478263 * ZZIIZIZZX
+ 0.029071696307478263 * IIIIIIIIX
+ 0.02810917628158917 * IIIIIIZXX
+ 0.02810917628158917 * IIIIIIIYY
+ 0.009047572463768893 * ZZIIZIZZZ
- 0.017379318062134313 * IIIIIIZXZ
- 0.017379318062134313 * ZZIIZIZXI
+ 0.03972517308420376 * ZIIZZIZII
+ 0.002870717885658319 * ZZIIZIYYI
- 0.0028707178856583115 * IZIZIIYYI
- 0.30666206888098657 * IIZZIIIII
- 0.3066620688809869 * IZZIIIIII
- 0.18779084636411902 * IIIIIIZZI
+ 0.31984940846548504 * IIIIIIZII
- 2.2158249643468846 * IIIIIZIII
+ 0.029071696307478165 * IIIIZXIII
+ 0.02907169630747818 * IIIIIXIII
+ 0.028109176281588977 * ZXXXXXIII
- 0.028109176281588977 * IYXXXYIII
- 0.009047572463768727 * IIIIZZIII
+ 0.017379318062134306 * ZXXXXZIII
+ 0.017379318062134306 * IYXXYIIII
- 0.03972517308420395 * IIIZZIIII
- 0.0028707178856582894 * XXXXZIIII
- 0.0028707178856582894 * YXXYIIIII
- 0.18779084636411938 * ZZIIIIIII
- 0.31984940846548493 * ZIIIIIIII
-